In [7]:
import base64
import pandas as pd
import io
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
from dash.dependencies import State

In [2]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pile_module #custom module

# Load CSV with all the H-Pile info
H_Pile = pd.read_csv("https://raw.githubusercontent.com/geotechnick/Geotechnical_Design/main/Pile_Design/H_Piles/HP_Spec_Table.csv")
pile_names = H_Pile['Section_E'].unique().tolist()

# Filter out the metric values
H_Pile = H_Pile.loc[:, ~H_Pile.columns.str.endswith('_M')]

# Create drop down with the pile names
dropdown = dcc.Dropdown(id='pile-dropdown', options=[{'label': name, 'value': name} for name in pile_names], value=pile_names[0])

# Define app layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Pile Information"),
    dropdown,
    html.Div(id='pile-info'),

    html.H1("File Upload"),
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        multiple=False
    ),
    html.Div(id='output-data-upload'),

    html.H1("Pile Axial Information"),
    
    html.Label("Ground Surface:"),
    dcc.Input(id="ground-surface", type="text", value=""),
    
    html.Label("Top Pile:"),
    dcc.Input(id="top-pile", type="text", value=""),
    
    html.Label("Pile Length:"),
    dcc.Input(id="pile-length", type="number", value=0),
    
    html.Label("Predrill Depth:"),
    dcc.Input(id="predrill-depth", type="number", value=0),
    
    html.Label("Steel Strength:"),
    dcc.Input(id="steel-strength", type="number", value=0),
    
    html.Button('Submit', id='submit-val', n_clicks=0),
    
    html.Div(id='output-data-submit'),

    ###test code####
    html.Button('Convert to JSON', id='convert-btn', n_clicks=0),
    html.Div(id='json-output-div')
])

# Callback to display pile info
@app.callback(
    Output('pile-info', 'children'),
    [Input('pile-dropdown', 'value')]
)
def display_pile_info(selected_value):
    return pile_module.display_row(selected_value, H_Pile)

# Callback to handle file upload
@app.callback(Output('output-data-upload', 'children'),
              [Input('upload-data', 'contents')],
              [State('upload-data', 'filename')])
def update_output(contents, filename):
    return pile_module.update_output(contents, filename)

# Callback to handle submission and create DataFrame
@app.callback(
    Output('output-data-submit', 'children'),
    [Input('submit-val', 'n_clicks')],
    [Input('ground-surface', 'value'),
     Input('top-pile', 'value'),
     Input('pile-length', 'value'),
     Input('predrill-depth', 'value'),
     Input('steel-strength', 'value')]
)
def create_dataframe(n_clicks, ground_surface, top_pile, pile_length, predrill_depth, steel_strength):
    return pile_module.create_dataframe(n_clicks, ground_surface, top_pile, pile_length, predrill_depth, steel_strength)

#test code########################
# Callback to convert dataframes to JSON upon button click
@app.callback(
    Output('json-output', 'data'),
    [Input('convert-btn', 'n_clicks')],
    [State('json-output', 'data')]
)
def convert_to_json(n_clicks, existing_data):
    if n_clicks > 0:
        # Convert dataframes to JSON
        json_data = {
            'Pile_Info': dict1,
            'Layer_Properties': dict2,
            'Pile_Axial_Info': dict3
        }
        return json_data
    else:
        # If button is not clicked yet, return existing data
        return existing_data

# Callback to display JSON output
@app.callback(
    Output('json-output-div', 'children'),
    [Input('json-output', 'data')]
)
def display_json_output(json_data):
    if json_data:
        # Display JSON output
        return html.Pre(str(json_data))
    else:
        return html.Div("No JSON data yet")
#test code########################

if __name__ == '__main__':
    app.run_server(debug=True)